In [1]:
!pip install qiskit qiskit_aer qiskit_ibm_runtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.8/366.8 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.8 MB/s eta 0:00:00


In [3]:
# QUBO with: (A) assignment-equality constraints, (B) capacity constraints with slack bits
import numpy as np
from itertools import permutations
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler

# -------------------------
# Data (distance matrix)
# -------------------------
distance_data = {
    "Hospital": {"Hospital": 0.0, "DT": 14.19, "GR": 17.78, "R2": 11.86, "R3_2": 7.34, "IT": 9.27},
    "DT": {"Hospital": 8.63, "DT": 0.0, "GR": 7.75, "R2": 19.67, "R3_2": 12.17, "IT": 9.4},
    "GR": {"Hospital": 11.5, "DT": 2.36, "GR": 0.0, "R2": 15.66, "R3_2": 10.05, "IT": 12.27},
    "R2": {"Hospital": 10.45, "DT": 11.92, "GR": 12.81, "R2": 0.0, "R3_2": 5.07, "IT": 8.32},
    "R3_2": {"Hospital": 10.85, "DT": 9.24, "GR": 10.12, "R2": 11.57, "R3_2": 0.0, "IT": 8.72},
    "IT": {"Hospital": 9.67, "DT": 9.43, "GR": 10.48, "R2": 11.54, "R3_2": 5.93, "IT": 0.0}
}

locations = ["Hospital", "DT", "GR", "R2", "R3_2", "IT"]
n_locations = len(locations)

# numpy distance matrix
dist_matrix = np.zeros((n_locations, n_locations))
for i,a in enumerate(locations):
    for j,b in enumerate(locations):
        dist_matrix[i,j] = distance_data[a][b]
print("Distance matrix:\n", dist_matrix)

# -------------------------
# Variables and indexing
# -------------------------
patient_names = ["DT","GR","R2","R3_2","IT"]
n_patients = len(patient_names)   # 5
n_trips = 2                       # t = 0,1

# slack bits per trip: we'll use two bits per trip (weights 1 and 2) -> can represent slack 0..3
slack_bits_per_trip = 2

# variable ordering requested:
var_names = []
for p in patient_names:
    for t in range(n_trips):
        var_names.append(f"y_{p}_{t}")
# slack variables: s_{trip}_{k}
for t in range(n_trips):
    for k in range(slack_bits_per_trip):
        var_names.append(f"s_{t}_{k}")

n_vars = len(var_names)  # should be 14
print("\nNumber of binary variables:", n_vars)
print("Variables:", var_names)

# helper map name->index
var_index = {name:i for i,name in enumerate(var_names)}

# helper to get y index
def y_index(patient_idx, trip_idx):
    # patient_idx in 0..n_patients-1
    return patient_idx * n_trips + trip_idx

def s_index(trip_idx, k):
    return n_patients * n_trips + trip_idx * slack_bits_per_trip + k

# -------------------------
# Build QUBO
# -------------------------
def build_qubo_with_constraints(B_assign=2000.0, B_cap=800.0, caps=(3,2)):
    """
    Improved QUBO:
      - Objective: hospital->patient (diag) + pairwise intra-trip distances (off-diags)
      - Constraint A: B_assign * (sum_t y_{p,t} - 1)^2
      - Constraint B: B_cap * (sum_p y_{p,t} - cap - sum_k 2^k s_{t,k})^2
    """
    Q = np.zeros((n_vars, n_vars))

    # --- Objective: hospital -> patient (linear) + pairwise inside-trip (quadratic)
    # Linear: hospital->patient (add to diag)
    for p_idx, p_name in enumerate(patient_names):
        loc_idx = p_idx + 1
        hosp_to_p = dist_matrix[0, loc_idx]
        p_to_hosp = dist_matrix[loc_idx, 0]
        # add avg(hosp->p, p->hosp) so symmetry of directed distances doesn't break things
        hosp_round = 0.5 * (hosp_to_p + p_to_hosp)
        for t in range(n_trips):
            v = y_index(p_idx, t)
            Q[v, v] += hosp_round

    # Quadratic: pairwise cost within the same trip
    for t in range(n_trips):
        for p in range(n_patients):
            for q in range(p+1, n_patients):
                v_p = y_index(p, t)
                v_q = y_index(q, t)
                # add pairwise distance between patient p and q
                # we use symmetric dist between patient nodes (indices p+1, q+1)
                pair_dist = dist_matrix[p+1, q+1]
                Q[v_p, v_q] += pair_dist
                Q[v_q, v_p] += pair_dist

    # (optional) Normalize objective magnitudes so penalties meaningful:
    # compute current max abs objective entry to normalize objective scale
    objective_scale = np.max(np.abs(Q)) if np.max(np.abs(Q)) > 0 else 1.0
    # scale down objective to ~O(1) while penalties are O(1000)
    Q = Q / objective_scale

    # --- Constraint A: one-hot (exact) for each patient
    # B_assign * (sum_t y_{p,t} - 1)^2
    for p in range(n_patients):
        y_vars = [y_index(p, t) for t in range(n_trips)]
        # add coeff_i * coeff_j for all i,j in y_vars (coeffs=1)
        for i in y_vars:
            for j in y_vars:
                Q[i, j] += B_assign * (1.0 * 1.0)
        # linear part -2*B_assign*1 added to diagonals
        for i in y_vars:
            Q[i, i] += -2.0 * B_assign * 1.0

    # --- Constraint B: capacity with slack bits (as you requested)
    for t in range(n_trips):
        var_list = []
        coeffs = []
        # y vars coefficient +1
        for p in range(n_patients):
            v = y_index(p, t)
            var_list.append(v); coeffs.append(1.0)
        # slack bits: coefficient = -2^k
        for k in range(slack_bits_per_trip):
            s_v = s_index(t, k)
            weight = 2**k
            var_list.append(s_v); coeffs.append(-float(weight))
        cap = float(caps[t])
        # expand (sum coeff_i * x_i - cap)^2
        for ii, i_var in enumerate(var_list):
            for jj, j_var in enumerate(var_list):
                Q[i_var, j_var] += B_cap * (coeffs[ii] * coeffs[jj])
            # linear from -2*cap*coeff_i
            Q[i_var, i_var] += -2.0 * B_cap * (coeffs[ii] * cap)

    # symmetrize numerically
    Q = 0.5 * (Q + Q.T)
    return Q

# choose penalty strengths
B_assign = 2000.0
B_cap = 400.0
caps = (3,2)  # capacity trip0=3, trip1=2
Q = build_qubo_with_constraints(B_assign=B_assign, B_cap=B_cap, caps=caps)
print("\nQUBO shape:", Q.shape)

# Diagnostics: count linear & quadratic nonzero terms
linear_terms = np.count_nonzero(np.diag(Q))
quad_terms = np.count_nonzero(Q) - linear_terms
print("Number of linear terms:", linear_terms)
print("Number of quadratic terms:", quad_terms)

# -------------------------
# helper: energy and interpret
# -------------------------
def instance_energy(bitstring, Q):
    x = np.array(list(map(int, bitstring)), dtype=int)
    diag = np.diag(Q)
    Q_off = Q - np.diag(diag)
    return float(x @ Q_off @ x + diag @ x)

def interpret_with_slack(bitstring):
    """Return (assign, slack_values, validity_A, validity_B)
       assign: dict patient->trip (1/2) or None
       slack_values: dict trip->int value (decoded from slack bits)
       validity_A: True if each patient assigned exactly once
       validity_B: True if equality LHS = 0 holds for each trip (using slack bits)
    """
    bits = list(map(int, list(bitstring)))
    assign = {}
    validity_A = True
    # assignments
    for p in range(n_patients):
        b0 = bits[y_index(p,0)]
        b1 = bits[y_index(p,1)]
        s = b0 + b1
        if s == 1:
            assign[patient_names[p]] = 1 if b0==1 else 2
        else:
            assign[patient_names[p]] = None
            validity_A = False

    # slack decode & capacity equality check
    slack_values = {}
    validity_B = True
    for t in range(n_trips):
        sval = 0
        for k in range(slack_bits_per_trip):
            sval += bits[s_index(t,k)] * (2**k)
        slack_values[t] = sval
        sum_y = sum(bits[y_index(p,t)] for p in range(n_patients))
        lhs = sum_y - caps[t] - sval
        if lhs != 0:
            validity_B = False

    return assign, slack_values, validity_A, validity_B

# -------------------------
# small-TSP for ordering and trip distance
# -------------------------
def best_trip_distance_and_path(patient_idx_list):
    if len(patient_idx_list) == 0:
        return 0.0, []
    best = float('inf')
    best_perm = None
    for perm in permutations(patient_idx_list):
        d = 0.0
        d += dist_matrix[0, perm[0]]
        for i in range(len(perm)-1):
            d += dist_matrix[perm[i], perm[i+1]]
        d += dist_matrix[perm[-1], 0]
        if d < best:
            best = d
            best_perm = perm
    path_names = ["Hospital"] + [locations[i] for i in best_perm] + ["Hospital"]
    return best, path_names

def total_distance_and_paths_from_assign(assign):
    trips = {1: [], 2: []}
    name_to_idx = {name: i+1 for i,name in enumerate(patient_names)}
    for name, t in assign.items():
        if t in (1,2):
            trips[t].append(name_to_idx[name])
    tot = 0.0
    trip_paths = {}
    for t in (1,2):
        d, path = best_trip_distance_and_path(trips[t])
        tot += d
        trip_paths[t] = (d, path)
    return tot, trip_paths

# Convert QUBO -> Ising (for QAOA)
# -------------------------
def qubo_to_ising_terms(Q):
    n = Q.shape[0]
    terms = []
    for i in range(n):
        for j in range(i+1, n):
            J = 0.25*(Q[i,j] + Q[j,i])
            if abs(J) > 1e-12:
                terms.append({'coeff': J, 'pauli':'ZZ', 'qubits':[i,j]})
    for i in range(n):
        h = 0.5*Q[i,i]
        for j in range(n):
            if j!=i:
                h += 0.25*Q[i,j]
        if abs(h) > 1e-12:
            terms.append({'coeff': h, 'pauli':'Z', 'qubits':[i]})
    return terms

cost_terms = qubo_to_ising_terms(Q)
print(f"\nConverted to {len(cost_terms)} Ising terms (Z/ZZ).")

# -------------------------
# QAOA builder and execution
# -------------------------
def create_qaoa_circuit(num_qubits, cost_terms, p=1, params=None, measure=True):
    if params is None:
        params = [0.5]*(2*p)
    gammas = params[:p]; betas = params[p:]
    qc = QuantumCircuit(num_qubits, num_qubits)
    qc.h(range(num_qubits))
    for layer in range(p):
        gamma = gammas[layer]
        for term in cost_terms:
            c = term['coeff']
            if term['pauli']=='Z':
                q = term['qubits'][0]
                qc.rz(2*gamma*c, q)
            else:
                a,b = term['qubits']
                qc.rzz(2*gamma*c, a, b)
        for q in range(num_qubits):
            qc.rx(2*betas[layer], q)
    if measure:
        qc.barrier(range(num_qubits))
        qc.measure(range(num_qubits), range(num_qubits))
    return qc

def invert_counts(counts):
    return {k[::-1]:v for k,v in counts.items()}

def execute_counts(qc, backend, shots=100):
    job = backend.run(qc, shots=shots)
    res = job.result()
    counts = res.get_counts(qc)
    return invert_counts(counts)

def expected_energy_from_counts(counts, Q):
    tot = sum(counts.values())
    e = 0.0
    for bs,cnt in counts.items():
        e += instance_energy(bs, Q) * cnt
    return e / tot

# -------------------------
# Simulation with Aer
# -------------------------
print("\n=== Running Simulation with Aer ===")
backend_sim = Aer.get_backend('qasm_simulator')

p = 2
init = np.random.uniform(0, 2*np.pi, 2*p)
def qaoa_objective(theta):
    qc = create_qaoa_circuit(n_vars, cost_terms, p=p, params=theta, measure=True)
    counts = execute_counts(qc, backend_sim, shots=2000)
    return expected_energy_from_counts(counts, Q)

print("\nOptimizing QAOA parameters (this may take a short while)...")
res = minimize(qaoa_objective, init, method='COBYLA', options={'maxiter':100,'disp':True})
print("Optimization result:", res)

# Final QAOA run with simulator
opt_theta = res.x
qc_final = create_qaoa_circuit(n_vars, cost_terms, p=p, params=opt_theta, measure=True)
counts_final = execute_counts(qc_final, backend_sim, shots=5000)
sorted_counts = sorted(counts_final.items(), key=lambda x: instance_energy(x[0], Q))

print("\nTop measured bitstrings (Simulation):")
for bs,cnt in sorted_counts[:12]:
    e = instance_energy(bs, Q)
    assign, slack_vals, okA, okB = interpret_with_slack(bs)
    dist = None; trip_paths = None
    if okA and okB:
        dist, trip_paths = total_distance_and_paths_from_assign(assign)
    print(f"{bs}  count={cnt}  prob={cnt/sum(counts_final.values()):.4f}  energy={e:.7f}  validA={okA} validB={okB} dist={dist} slack={slack_vals}")
    if okA and okB:
        for t in sorted(trip_paths.keys()):
            d,path = trip_paths[t]
            print(f"   Trip {t}: { ' -> '.join(path) }  (dist {d:.2f} km)")

best_sim_bs = min(sorted_counts, key=lambda x: instance_energy(x[0], Q))[0]
assign_s, slack_s, okA_s, okB_s = interpret_with_slack(best_sim_bs)
if okA_s and okB_s:
    best_sim_dist, best_sim_paths = total_distance_and_paths_from_assign(assign_s)
    dist_str = f"{best_sim_dist:.2f} km"
else:
    best_sim_dist = None
    best_sim_paths = None
    dist_str = "None"
print(f"\nSimulation best-measured: bs={best_sim_bs} "
      f"energy={instance_energy(best_sim_bs,Q):.2f} "
      f"okA={okA_s} okB={okB_s} "
      f"assign={assign_s} slack={slack_s} distance={dist_str}")

# -------------------------
# Real Quantum Hardware Execution
# -------------------------
print("\n=== Running on Real Quantum Hardware ===")
try:
    # Initialize IBM Runtime Service
    service = QiskitRuntimeService(
        channel="ibm_quantum_platform",
        token="ZZK7WcXIp98apZIqPp_OCkQednMoG7HUGx9SSTMda7uK",
        instance="qui"
    )

    # Choose a real quantum backend
    backend_real = service.backend("ibm_brisbane")
    print(f"Using backend: {backend_real.name}")

    # Transpile the circuit for the target backend
    transpiled_qc = transpile(qc_final, backend=backend_real)
    print(f"Circuit transpiled for {backend_real.name}")

    # Run on real hardware with 1024 shots
    sampler = Sampler(mode=backend_real)
    job = sampler.run([transpiled_qc], shots=1024)
    print(f"Job submitted: {job.job_id()}")

    # Wait for the job to complete
    result = job.result()
    counts_real = result[0].data.c.get_counts()

    # Invert counts to match our convention
    counts_real = invert_counts(counts_real)

    # Process and display results
    sorted_counts_real = sorted(counts_real.items(), key=lambda x: instance_energy(x[0], Q))

    print("\nTop measured bitstrings (Real Hardware):")
    for bs, cnt in sorted_counts_real[:12]:
        e = instance_energy(bs, Q)
        assign, slack_vals, okA, okB = interpret_with_slack(bs)
        dist = None; trip_paths = None
        if okA and okB:
            dist, trip_paths = total_distance_and_paths_from_assign(assign)
        print(f"{bs}  count={cnt}  prob={cnt/sum(counts_real.values()):.4f}  energy={e:.7f}  validA={okA} validB={okB} dist={dist} slack={slack_vals}")
        if okA and okB:
            for t in sorted(trip_paths.keys()):
                d,path = trip_paths[t]
                print(f"   Trip {t}: { ' -> '.join(path) }  (dist {d:.2f} km)")

    best_real_bs = min(sorted_counts_real, key=lambda x: instance_energy(x[0], Q))[0]
    assign_r, slack_r, okA_r, okB_r = interpret_with_slack(best_real_bs)
    print(f"okA_r:{okA_r},okB_r:{okB_r}")
    if okA_r and okB_r:
        best_real_dist, best_real_paths = total_distance_and_paths_from_assign(assign_r)
        dist_str = f"{best_real_dist:.2f} km"
    else:
        best_real_dist = None
        best_real_paths = None
        dist_str = "None"
    print(f"\nReal hardware best-measured: bs={best_real_bs} "
          f"energy={instance_energy(best_real_bs,Q):.2f} "
          f"okA={okA_r} okB={okB_r} "
          f"assign={assign_r} slack={slack_r} distance={dist_str}")

    # Compare simulation and real hardware results
    print("\n=== Comparison ===")
    print(f"Simulation best distance: {best_sim_dist:.2f} km")
    print(f"Real hardware best distance: {best_real_dist:.2f} km")
    if best_sim_dist and best_real_dist:
        print(f"Difference: {abs(best_sim_dist - best_real_dist):.2f} km")

except Exception as e:
    print(f"Error running on real hardware: {str(e)}")
    print("Skipping real hardware execution...")

Distance matrix:
 [[ 0.   14.19 17.78 11.86  7.34  9.27]
 [ 8.63  0.    7.75 19.67 12.17  9.4 ]
 [11.5   2.36  0.   15.66 10.05 12.27]
 [10.45 11.92 12.81  0.    5.07  8.32]
 [10.85  9.24 10.12 11.57  0.    8.72]
 [ 9.67  9.43 10.48 11.54  5.93  0.  ]]

Number of binary variables: 14
Variables: ['y_DT_0', 'y_DT_1', 'y_GR_0', 'y_GR_1', 'y_R2_0', 'y_R2_1', 'y_R3_2_0', 'y_R3_2_1', 'y_IT_0', 'y_IT_1', 's_0_0', 's_0_1', 's_1_0', 's_1_1']

QUBO shape: (14, 14)
Number of linear terms: 14
Number of quadratic terms: 94

Converted to 61 Ising terms (Z/ZZ).

=== Running Simulation with Aer ===

Optimizing QAOA parameters (this may take a short while)...
Return from COBYLA because the trust region radius reaches its lower bound.
Number of function values = 37   Least value of F = -7021.185771352314
The corresponding X is:
[2.77737658 2.37449078 2.9865802  2.83406876]

Optimization result:  message: Return from COBYLA because the trust region radius reaches its lower bound.
 success: True
  status: